In [35]:
from pprint import pprint
from typing import Any

from pydantic import BaseModel, RootModel
from pydantic_ai import Agent

agent = Agent("openai:gpt-4o-mini", name="test_case_generator", retries=1)

@agent.system_prompt
def test_case_system_prompt():
    return """You are a helpful test case generator. You will be given a description of functionality 
    and should generate test cases that thoroughly validate the described behavior.
    
    <TEST CASE OUTPUT FORMAT>
    name: <name of the test case>
    description: <description of the test case>
    input: <input values for the test case>
    expected_output: <expected output/behavior of the test case>
    preconditions: <any relevant preconditions for the test case>
    </TEST CASE OUTPUT FORMAT>

    <OUTPUT FORMAT: list of dictionaries>
    [<TEST CASE OUTPUT FORMAT>, <TEST CASE OUTPUT FORMAT>, ...]
    </OUTPUT FORMAT>

    Be aware of the number of test casaes the user wants and output the correct number of test cases in the correct output format.
    """

agent.result_validator = lambda x: isinstance(x, list)

In [25]:
api_spec = {
    "paths": {
        "/pets": {
            "get": {
                "parameters": [
                    {
                        "name": "status",
                        "in": "query",
                        "type": "string",
                        "required": True,
                    }
                ]
            },
            "post": {
                "requestBody": {
                    "content": {
                        "application/json": {
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "name": {"type": "string"},
                                    "age": {"type": "integer"},
                                },
                            }
                        }
                    }
                }
            },
        }
    }
}

In [26]:
response = await agent.run("Generate one case for this API spec: " + str(api_spec))


In [46]:
class TestCase(BaseModel):
    name: str
    description: str
    input_json: dict[str, Any] | list[dict[str, Any]] | None = None
    expected_output_prompt: str | None = None
    expected_output_json: dict[str, Any] | list[dict[str, Any]] | None = None
    preconditions: str | None = None

class SuiteTestCases(RootModel[list[TestCase]]):
    pass

test_cases = SuiteTestCases.model_validate_json(response.data)

pprint(test_cases.model_dump())

[{'description': 'Test retrieving a list of pets with a valid status',
  'expected_output_json': None,
  'expected_output_prompt': None,
  'input_json': None,
  'name': 'GetPetsByStatus',
  'preconditions': 'The API server is running and has pets with status '
                   "'available' in the database."}]


## Planning steps 

In [ ]:
planning_agent = Agent("openai:gpt-4o-mini", name="test_case_generator", retries=1)

@agent.system_prompt
def test_case_system_prompt():
    return """You are a helpful test case generator. You will be given a description of functionality 
    and should generate test cases that thoroughly validate the described behavior.
    
    <TEST CASE OUTPUT FORMAT>
    name: <name of the test case>
    description: <description of the test case>
    input: <input values for the test case>
    expected_output: <expected output/behavior of the test case>
    preconditions: <any relevant preconditions for the test case>
    </TEST CASE OUTPUT FORMAT>

    <OUTPUT FORMAT: list of dictionaries>
    [<TEST CASE OUTPUT FORMAT>, <TEST CASE OUTPUT FORMAT>, ...]
    </OUTPUT FORMAT>

    Be aware of the number of test casaes the user wants and output the correct number of test cases in the correct output format.
    """

agent.result_validator = lambda x: isinstance(x, list)